#Building a simple Keras + deep learning REST API

In this tutorial, we will present a simple method to take a Keras model and deploy it as a REST API.

The examples covered in this post will serve as a template/starting point for building your own deep learning APIs — you will be able to extend the code and customize it based on how scalable and robust your API endpoint needs to be.

Specifically, we will learn:

How to (and how not to) load a Keras model into memory so it can be efficiently used for inference
How to use the Flask web framework to create an endpoint for our API
How to make predictions using our model, JSON-ify them, and return the results to the client
How to call our Keras REST API using both cURL and Python
By the end of this tutorial you'll have a good understanding of the components (in their simplest form) that go into a creating Keras REST API.

Feel free to use the code presented in this guide as a starting point for your own deep learning REST API.

#Configuring your development environment

We'll be making the assumption that Keras is already configured and installed on your machine. If not, please ensure you install Keras using the official install instructions.

From there, we'll need to install Flask (and its associated dependencies), a Python web framework, so we can build our API endpoint. We'll also need requests so we can consume our API as well.

The relevant pip install commands are listed below:

In [ ]:
!pip install flask gevent requests pillow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#CREATE A Procfile : Create a file with name Procfile and paste content below line in it into it



web: gunicorn app:app

In [ ]:
procfile = 'web: gunicorn app:app'
procfiles= open("/content/Procfile","w")
procfiles.write(procfile)
procfiles.close()

#INSTALL FLASK AND NGROK

In [ ]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask

#Build front end

In [ ]:
a = '''
<!doctype html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.1/jquery.min.js" type="text/javascript"></script>
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js" type="text/javascript"></script>
  <title>Image Recognition Server</title>
  <body class style="margin:10px;padding:10px">
    <div class="page-header" id="banner">
      <div class="row">
        <div class="col-lg-8 col-md-7 col-sm-6">
          <h3>Image Recognition Server</h3>
          <p class="lead">Upload the image and find out what an animal is located on it?</p>
        </div>
      </div>
      <form action="" method=post enctype=multipart/form-data>
        <input type=file name=file>
        <input type=submit value=Upload>
    </form>
    </div>
    <p style="margin-bottom:2cm;"></p>
    <div class="row">
        <div class="col-lg-4">
          <div class="page-header">
            <h3 id="tables">Result</h3>
          </div>
          <div class="bs-component">
            <table class="table table-hover">
                <tr class="table-active">
                 
                  <th scope="col">Predict</th>
                </tr>
                <tr>
                         <td> {{label}} </td>
                  </tr>
            </table> 
        </div>
      </div>
  </body>
  '''

#Build directory structure

In [ ]:
!mkdir '/content/templates'
!mkdir '/content/uploads'

In [ ]:
Html_file= open("/content/templates/index.html","w")
Html_file.write(a)
Html_file.close()

#Building your Keras REST API

Our Keras REST API is self-contained in a single file named run_keras_server.py. We kept the installation in a single file as a manner of simplicity — the implementation can be easily modularized as well.

Inside run_keras_server.py you'll find three functions, namely:

1. load_model: Used to load our trained Keras model and prepare it for inference.
2. prepare_image: This function preprocesses an input image prior to passing it through our network for prediction. If you are not working with image data you may want to consider changing the name to a more generic prepare_datapoint and applying any scaling/normalization you may need.
3. predict: The actual endpoint of our API that will classify the incoming data from the request and return the results to the client.

In [ ]:
!pip install -U efficientnet

In [ ]:
import os
from flask import Flask, jsonify, request, redirect, url_for, send_from_directory, render_template
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model
from werkzeug.utils import secure_filename
import numpy as np
# from keras.applications import VGG16
# from keras.applications.vgg16 import preprocess_input, decode_predictions
# from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetB7
from keras.preprocessing import image

ALLOWED_EXTENSIONS = set(['jpg', 'jpeg', 'png'])
IMAGE_SIZE = (640, 640)
UPLOAD_FOLDER = 'uploads'
model = None
app = Flask(__name__)
run_with_ngrok(app)
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

class_names = [
  'Приора', #0
  'Ford Focus', #1
  'Самара', #2
  'ВАЗ-2110', #3
  'Жигули', #4
  'Нива', #5
  'Калина', #6
  'ВАЗ-2109', #7
  'Volkswagen Passat', #8
  'ВАЗ-21099' #9
]

def load_model():
		# load the pre-trained Keras model (here we are using a model
    # pre-trained on ImageNet and provided by Keras, but you can
    # substitute in your own networks just as easily)
    global model
    # model = VGG16(weights="imagenet")
    model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/deep learning/best_model_en_512_2_3.hdf5')

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

def predict(file):
    img  = image.load_img(file, target_size=IMAGE_SIZE)
    img = np.array(img)
    prediction = model.predict(img[np.newaxis, ...]/255)[0].argmax()
    print(prediction)
    return class_names[prediction]
    
    # img  = image.load_img(file, target_size=IMAGE_SIZE)
    # img = img_to_array(img)
    # img = np.expand_dims(img, axis=0)
    # img = preprocess_input(img)
    # probs = model.predict(img)
    # print(probs)
    # print(probs.argmax(axis=-1))
    # probs = probs.argmax(axis=-1)
    # predict = probs.tolist()
    # print(predict[0])
    # return class_names[predict[0]]
   
    # topfive_labels = " "
    # for (imagenetID, label, prob) in decode_predictions(probs, top=5)[0]:
    #   topfive_labels += str(label+", ")
    # return topfive_labels

@app.route("/")
def template_test():
    return render_template('index.html', label='', imagesource='file://null')

@app.route('/', methods=['POST'])
def upload_file():
    
        file = request.files['file']
        
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)
            output = predict(file_path)
        return render_template("index.html", label=output)

if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server...please wait until server has fully started"))
    load_model()
    app.run()